In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import seaborn as sbs
import numpy as np

spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "4g")
    .config("spark.executer.memory", "8g")
    .getOrCreate()
)

22/09/08 10:43:00 WARN Utils: Your hostname, Luo resolves to a loopback address: 127.0.1.1; using 172.30.203.116 instead (on interface eth0)
22/09/08 10:43:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/08 10:43:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/08 10:43:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
trans = '../data/curated/transaction cleaned.parquet'
trans_full = '../data/curated/transaction cleaned without remove outlier.parquet/'
merchant = '../data/curated/merchant.csv'

In [3]:
trans_sdf = spark.read.parquet(trans)
trans_full_sdf = spark.read.parquet(trans_full)
merchant_sdf = spark.read.csv(merchant, header = True)

In [4]:
merchant_sdf.count()

4026

In [5]:
trans_sdf.count()

7813531

In [4]:
merged_full_sdf = trans_full_sdf.join(merchant_sdf,trans_full_sdf.merchant_abn ==  merchant_sdf.merchant_abn,"rightouter").drop(merchant_sdf.merchant_abn)

In [5]:
merged_sdf = trans_sdf.join(merchant_sdf,trans_sdf.merchant_abn ==  merchant_sdf.merchant_abn,"rightouter").drop(merchant_sdf.merchant_abn)

In [7]:
merged_sdf = merged_sdf.withColumn('revenue', F.col('dollar_value') * F.col('take_rate') /100)

In [ ]:
merged_sdf

In [9]:
merged_sdf.write.mode('overwrite').parquet("../data/curated/transaction merge merchant.parquet")

In [9]:
missing_full = merged_full_sdf.where(F.col('dollar_value').isNull())

In [10]:
missing_full.count()

7

In [6]:
missing = merged_sdf.where(F.col('dollar_value').isNull())

In [11]:
missing.count()

197

22/09/08 11:05:32 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1024410 ms exceeds timeout 120000 ms
22/09/08 11:05:32 WARN SparkContext: Killing executors is not supported by current scheduler.


In [12]:
missing.count()

197